# 新闻文本分析
首先我们将文本数据读取到编译器中

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np

train_df = pd.read_csv('data/train_set.csv', sep='\t')
test_df = pd.read_csv('data/test_a.csv', sep='\t')
train_df.head()

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...


进行文本的分类，最简单的，也是最符合直观想法的就是统计每一个词的词频，根据不同词汇出现频率的差异来决定文档所属的类别。比如出现“股票”，“上市”，“分红”等词汇的新闻通常就是财经类新闻，而出现“飞机”，“大炮”，“坦克”这类词汇的新闻就基本肯定是军事类新闻了。所以，挑选出具有分辨力的词汇对于文本分类工作就是非常重要的。

首先我们去统计不同词汇的出现频率。由于这里的文本已经变成了数字，没有办法通过中文分词的方式来进行词汇切分。所以我们默认词汇由两个字或者三个字构成。sklearn中的CountVectorizer类可以帮助我们迅速地统计文本中不同词汇的词频。

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
# ngram_range参数管理了几个字符构成一个词汇，这里的range(2,4)就是认为2或者3个字符构成一个词汇
# max_features参数管理了一个挑选出多少个关键词，该方法内部有特定的挑选原则，暂时先不用关注
# 如果内存不够的话，可以把2000变成1000
vectorizer = CountVectorizer(ngram_range=range(2, 4), max_features=2000)
train_sample_num = 180000
# 基于前50000个样本构建词频统计模型
vectorizer.fit(train_df['text'].iloc[:50000])
train_x = vectorizer.transform(train_df['text'])

fit方法是为了基于一部分样本去估计出哪一些词汇对于分类更重要，选定出max_features个词汇。而transform则是将整个数据集按照这个词汇集合，统计出每一个目标词汇的出现次数，然后将文本转换成这个词频列表。CountVectorizer的具体使用方法你可以查阅[该文档](https://blog.csdn.net/weixin_38278334/article/details/82320307)

In [12]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

x_to_train = train_x[:train_sample_num]
x_to_valid = train_x[train_sample_num:]
y_to_train = train_df['label'].values[:train_sample_num]
y_to_valid = train_df['label'].values[train_sample_num:]

clf = RidgeClassifier(alpha=0.1)
clf.fit(x_to_train, y_to_train)
print('训练集内的预测准确率为：%.4f' % clf.score(x_to_train, y_to_train))
print('验证集中的预测准确率为：%.4f' % clf.score(x_to_valid, y_to_valid))
y_to_valid_pred = clf.predict(x_to_valid)
valid_F1_valid = f1_score(y_to_valid, y_to_valid_pred, average='macro')
print('在验证集上的F1得分为：%.4f' % valid_F1_valid)

训练集内的预测准确率为：0.8523
验证集中的预测准确率为：0.8442
在验证集上的F1得分为：0.7442


多分类问题上，由于不同类别数量上的不平衡，所以不宜使用单纯的正确率来衡量分类器的性能，而应该使用Macro-F1这种评价指标。具体来说，该指标的计算方法可以查看[该文档](https://blog.csdn.net/qq280929090/article/details/70598629)。

在CountVectorizer类中，还有两个重要的参数可以设置以改善特征的筛选效果。我们知道，有一些词语是语言中的结构性词汇，比如“还是”，“并且”等等，这类词汇并没有任何实际意义，只是为了满足语言的语法结构，所以对于文本分类是没有用处的。这类词汇的特点是在各种句子中都会出现，我们特意地为这一现象定义一个概念，叫做文档频率（Document Frequency, df），即在一个文本集合中共有N个文档，某一词汇在其中的n个文档中出现过，那么该词汇的文档频率就是`n/N`。显然结构性词汇的文档频率就会很高。我们要筛选掉那些文档频率df很大的词汇。反之，如果某一个词汇的文档频率过低，只在几篇文档中出现过，那么这样的词汇也没必要作为特征词汇，因为它不具有推广意义。我们通过`max_df`和`min_df`在CountVectorizer类中设置文档频率的允许最大值和最小值。

In [15]:
vectorizer = CountVectorizer(ngram_range=range(2, 4), max_features=2000, max_df=0.3, min_df=0.001)
train_sample_num = 180000
# 基于前50000个样本构建词频统计模型
vectorizer.fit(train_df['text'].iloc[:50000])
train_x = vectorizer.transform(train_df['text'])

x_to_train = train_x[:train_sample_num]
x_to_valid = train_x[train_sample_num:]
y_to_train = train_df['label'].values[:train_sample_num]
y_to_valid = train_df['label'].values[train_sample_num:]

clf = RidgeClassifier(alpha=0.1)
clf.fit(x_to_train, y_to_train)
print('训练集内的预测准确率为：%.4f' % clf.score(x_to_train, y_to_train))
print('验证集中的预测准确率为：%.4f' % clf.score(x_to_valid, y_to_valid))
y_to_valid_pred = clf.predict(x_to_valid)
valid_F1_valid = f1_score(y_to_valid, y_to_valid_pred, average='macro')
print('在验证集上的F1得分为：%.4f' % valid_F1_valid)

训练集内的预测准确率为：0.8518
验证集中的预测准确率为：0.8438
在验证集上的F1得分为：0.7448


显然，修改了这两个参数之后，F1得分稍微增加了一点，如何更好的设置这两个参数需要多长试验来确定。

CountVectorizer方法只是统计某一些关键词的词频来决定样本的类别，而真正更有效的词汇应该是在全局文档集合中出现的概率不大，但是在该文档中出现的频率很大，比如“同时”这个词，它不仅在文档A中出现的次数很多，同时在全部的文档中都出现了，那么这一个词汇就不那么重要。而“坦克”这一个词汇在文档A中出现的次数没有“同时”这个词汇多，但是由于“坦克”在其他文档中出现的次数很少，所以“坦克”这个词对于确定文档A的类别就要比“同时”这个词更为重要。这种思想演化出来一种算法叫做“TF-IDF”算法，中文叫做“词频-逆向文档频率”算法，具体的计算方法可以查阅[该文档](https://zhuanlan.zhihu.com/p/31197209)。

sklearn中有简单的TF-IDF算法实现。使用方法如下：

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range=(2,4), max_features=2000)
tfidf.fit(train_df['text'].iloc[:50000])
train_x = tfidf.transform(train_df['text'])

x_to_train = train_x[:train_sample_num]
x_to_valid = train_x[train_sample_num:]
y_to_train = train_df['label'].values[:train_sample_num]
y_to_valid = train_df['label'].values[train_sample_num:]

clf = RidgeClassifier(alpha=0.1)
clf.fit(x_to_train, y_to_train)
print('训练集内的预测准确率为：%.4f' % clf.score(x_to_train, y_to_train))
print('验证集中的预测准确率为：%.4f' % clf.score(x_to_valid, y_to_valid))
y_to_valid_pred = clf.predict(x_to_valid)
valid_F1_valid = f1_score(y_to_valid, y_to_valid_pred, average='macro')
print('在验证集上的F1得分为：%.4f' % valid_F1_valid)

训练集内的预测准确率为：0.9005
验证集中的预测准确率为：0.8973
在验证集上的F1得分为：0.8507


无论是CountVectorizer这种寻找高频词的词频作为特征的模型，还是TF-IDF这种选择tfidf得分作为特征的模型，都是先选择出一组词汇，然后将文本转换为这一组词汇上的统计指标，比如词频或者tfidf得分。词汇与词汇之间是没有关联的。如果能够有效利用词汇之间的语义关联，那么就可以更有效的进行文本分类。这种语义关联是什么呢？比如儿童和孩子就是同义词，这两个在统计词频或者tfidf得分的时候就应该视为同一个词，而男人与国王的差距应该和女人与王后的差距是一样的。所以，如果可以把词汇转换成一个向量，这个向量反映了词汇的各方面语义，那么基于这个向量去进行分类，就会得到很好的结果，我们把这种从词汇到语义向量的映射称之为词嵌入。具体的操作可以查阅[该文档](https://blog.csdn.net/puredreammer/article/details/78330821)。

python上的fasttext模块可以快速的进行基于词嵌入的文本分类。运行`pip install fasttext`安装这个模块。

In [8]:
# fasttext模块读取数据的格式有特殊要求，所以要把数据样式变换一下
# fasttext模块在训练的时候会之间从一个文本文件中读取数据，不需要从内存从传入，所以要把一部分训练数据转存为文本文件
# fasttext要求这个数据文件的格式为
"""
文本内容, __label__1
文本内容，__label__3
文本内容，__label__5
...
"""
# 这一步是在每一个label数字前面追加一个"__label__"
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
# 选择前180000条样本写入到一个文件中
train_sample_num = 180000
train_df[['text','label_ft']].iloc[:train_sample_num].to_csv('train.csv', index=None, header=None, sep='\t')

# 使用这一个训练数据文件对fasttext模型进行训练
import fasttext
model = fasttext.train_supervised('train.csv', lr=1.0, wordNgrams=2, 
                                  verbose=2, minCount=1, epoch=25, loss="hs")

x_to_valid = train_df.iloc[train_sample_num:]['text']
y_to_valid = train_df['label'].values[train_sample_num:].astype(str)
# fasttext给出的文本类别预测结果也是如__label__3这种形式，所以，需要把结果裁剪一下。
y_to_valid_pred = [model.predict(x)[0][0].split('__')[-1] for x in x_to_valid]
print('在验证集上的F1得分为：%.4f' % f1_score(y_to_valid, y_to_valid_pred, average='macro'))

在验证集上的F1得分为：0.9174


可以看到结果，FastText使用比较简单，对数据进行简单转换之后，放到模型中，就能够得到很好的结果。运行速度也比较快，这就是词嵌入工具的良好效果。接下来我们对测试集进行测试，并上传最终结果。

In [9]:
x_to_test = test_df['text']
y_to_test_pred = [model.predict(x)[0][0].split('__')[-1] for x in x_to_test]
with open('outcome.csv', 'w') as f:
    f.write('label\n')
    for i in y_to_test_pred:
        f.write(str(i) + '\n')

在线上我们得到了0.9125的成绩，还是一个不错的结果。这就是文本分类三种简单方法，在10个不同的类别上得到了高于90%的分类正确率，在大部分场景下应该是足够高的分类准确率了。